## Problem Set 11

First the exercise:
* What is the maximum depth of a decision tree trained on $N$ samples?
The decision tree must make a proper split at each node, so the size of each node must reduce by at least one as we move down one level. So the maximum depth of a  decision tree is $N-1$.
* If we train a decision tree to an arbitrary depth, what will be the training error?
Assuming the training data assigns unique labels to samples with identical features, this will be Zero. If we train a decision tree to arbitrary depth we will end up with a tree where each node contains samples with identical features. If each of these samples has the same label than any of the standard rules (voting, averaging) will return the correct response.
* How can we alter a loss function to help regularize a decision tree?
One of the simplest ways is to add to our loss function an increasing function of the depth of the node. For example, we could just add $\lambda |D|$ or perhaps $\lambda 2^|D|$ where $\lambda$ is an appropriate hyperparameter (probably very small). One should choose so that growth of this regularization term so that it will not dominate the unregularized cost function when obtaining improvements at the desired rate. 

### Python Lab

Now let us load our standard libraries.

In [1]:
import numpy as np
import pandas as pd

Let us load the credit card dataset and extract a small dataframe of numerical features to test on.

In [26]:
big_df = pd.read_csv("UCI_Credit_Card.csv")

In [28]:
big_df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [29]:
len(big_df)

30000

In [30]:
len(big_df.dropna())

30000

In [31]:
df = big_df.drop(labels = ['ID'], axis = 1)

In [40]:
labels = df['default.payment.next.month']
df.drop('default.payment.next.month', axis = 1, inplace = True)

In [41]:
num_samples = 25000

In [42]:
train_x, train_y = df[0:num_samples], labels[0:num_samples]

In [43]:
test_x, test_y = df[num_samples:], labels[num_samples:]

In [48]:
test_x.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
25000,410000.0,1,1,1,38,-1,-1,-1,-1,-2,...,35509.0,0.0,0.0,0.0,0.0,35509.0,0.0,0.0,0.0,0.0
25001,260000.0,1,2,2,35,0,0,0,0,0,...,297313.0,276948.0,2378.0,-2709.0,12325.0,6633.0,6889.0,1025.0,2047.0,194102.0
25002,50000.0,1,2,1,40,0,0,0,0,0,...,11353.0,12143.0,11753.0,11922.0,1200.0,4000.0,2000.0,2000.0,1000.0,1000.0
25003,360000.0,1,3,1,37,-1,-1,-1,-2,-2,...,0.0,0.0,0.0,0.0,303.0,0.0,0.0,0.0,0.0,860.0
25004,50000.0,1,3,1,49,0,0,0,0,0,...,50076.0,48995.0,19780.0,15102.0,2000.0,5000.0,2305.0,3000.0,559.0,3000.0


In [114]:
train_y.head()

0    1
1    1
2    0
3    0
4    0
Name: default.payment.next.month, dtype: int64

Now let us write our transformation function.

In [196]:
class bin_transformer(object):
    
    def __init__(self, df, num_quantiles = 2):
        self.quantiles = [None]*(num_quantiles-1)
        for ix,q in enumerate(np.linspace(1./num_quantiles,1.,num_quantiles-1)):
            self.quantiles[ix] = df.quantile(q)
            
    
    def transform(self, df):
        new = pd.DataFrame()
        fns = {}
        for col_name in df.axes[1]:
            for ix, q in enumerate(self.quantiles):
                quart = q[col_name]
                new[col_name+str(ix)] = (df[col_name] >= quart)
                fns[col_name+str(ix)] =(col_name, lambda x: x[col_name]>=quart)
        return new, fns

In [176]:
transformer = bin_transformer(df,5)

In [178]:
train_x_t, tr_fns = transformer.transform(train_x)

In [179]:
test_x_t, test_fns = transformer.transform(test_x)

In [181]:
train_x_t.head()

,LIMIT_BAL0,LIMIT_BAL1,LIMIT_BAL2,LIMIT_BAL3,SEX0,SEX1,SEX2,SEX3,EDUCATION0,EDUCATION1,...,PAY_AMT42,PAY_AMT43,PAY_AMT50,PAY_AMT51,PAY_AMT52,PAY_AMT53,PAY_AMT60,PAY_AMT61,PAY_AMT62,PAY_AMT63
0,False,False,False,False,True,True,True,True,True,True,...,False,False,True,False,False,False,True,False,False,False
1,True,False,False,False,True,True,True,True,True,True,...,False,False,True,False,False,False,True,True,False,False
2,True,False,False,False,True,True,True,True,True,True,...,False,False,True,False,False,False,True,True,True,False
3,True,False,False,False,True,True,True,True,True,True,...,False,False,True,False,False,False,True,False,False,False
4,True,False,False,False,True,False,False,False,True,True,...,True,False,True,False,False,False,True,False,False,False


In [182]:
tr_fns

{'AGE0': ('AGE',
  <function __main__.bin_transformer.transform.<locals>.<lambda>>),
 'AGE1': ('AGE',
  <function __main__.bin_transformer.transform.<locals>.<lambda>>),
 'AGE2': ('AGE',
  <function __main__.bin_transformer.transform.<locals>.<lambda>>),
 'AGE3': ('AGE',
  <function __main__.bin_transformer.transform.<locals>.<lambda>>),
 'BILL_AMT10': ('BILL_AMT1',
  <function __main__.bin_transformer.transform.<locals>.<lambda>>),
 'BILL_AMT11': ('BILL_AMT1',
  <function __main__.bin_transformer.transform.<locals>.<lambda>>),
 'BILL_AMT12': ('BILL_AMT1',
  <function __main__.bin_transformer.transform.<locals>.<lambda>>),
 'BILL_AMT13': ('BILL_AMT1',
  <function __main__.bin_transformer.transform.<locals>.<lambda>>),
 'BILL_AMT20': ('BILL_AMT2',
  <function __main__.bin_transformer.transform.<locals>.<lambda>>),
 'BILL_AMT21': ('BILL_AMT2',
  <function __main__.bin_transformer.transform.<locals>.<lambda>>),
 'BILL_AMT22': ('BILL_AMT2',
  <function __main__.bin_transformer.transform.<l

Now let us build some simple loss functions for 1d labels.

In [160]:
def bdd_cross_entropy(pred, label):
    return -np.mean(label*np.log(pred+10**(-20)))

In [161]:
def MSE(pred,label):
    return np.mean((pred-label)**2)

In [162]:
def acc(pred,label):
    return np.mean((pred>=0.5)==(label == 1))

Now let us define the find split function.

In [184]:
def find_split(x, y, loss, verbose = False):
    min_ax = None
    base_loss = loss(np.mean(y),y) 
    min_loss = base_loss
    N = len(x)
    for col_name in x.axes[1]:
        mask = x[col_name]
        num_pos = np.sum(mask)
        num_neg = N - num_pos
        pos_y = np.mean(y[mask])
        neg_y = np.mean(y[~mask])
        l = (num_pos*loss(pos_y, y[mask]) + num_neg*loss(neg_y, y[~mask]))/N
        if verbose:
            print("Column {0} split has improved loss {1}".format(col_name, base_loss-l))
        if l < min_loss:
            min_loss = l
            min_ax = col_name
    return min_ax, min_loss
        

In [185]:
find_split(train_x_t, train_y, MSE, verbose = True)

Column LIMIT_BAL0 split has improved loss 0.0032026111833937665
Column LIMIT_BAL1 split has improved loss 0.0038879427292773105
Column LIMIT_BAL2 split has improved loss 0.0022416992324077734
Column LIMIT_BAL3 split has improved loss 1.991381031213324e-06
Column SEX0 split has improved loss nan
Column SEX1 split has improved loss 0.0002155159725325817
Column SEX2 split has improved loss 0.0002155159725325817
Column SEX3 split has improved loss 0.0002155159725325817
Column EDUCATION0 split has improved loss 2.3907091916103296e-05
Column EDUCATION1 split has improved loss 0.0004640208803457502
Column EDUCATION2 split has improved loss 0.0004640208803457502
Column EDUCATION3 split has improved loss 1.6813173923713176e-05
Column MARRIAGE0 split has improved loss 3.249086770407139e-05
Column MARRIAGE1 split has improved loss 0.00014480802024710582
Column MARRIAGE2 split has improved loss 0.00014480802024710582
Column MARRIAGE3 split has improved loss 1.6656260906328102e-05
Column AGE0 split

('LIMIT_BAL1', 0.16944952287072268)

In [186]:
find_split(train_x_t, train_y, bdd_cross_entropy, verbose = True)

Column LIMIT_BAL0 split has improved loss 0.006244357785518517
Column LIMIT_BAL1 split has improved loss 0.008711301876880462
Column LIMIT_BAL2 split has improved loss 0.005472328942544402
Column LIMIT_BAL3 split has improved loss 8.924978500690628e-06
Column SEX0 split has improved loss nan
Column SEX1 split has improved loss 0.000478882743748521
Column SEX2 split has improved loss 0.000478882743748521
Column SEX3 split has improved loss 0.000478882743748521
Column EDUCATION0 split has improved loss 0.00010712331165202427
Column EDUCATION1 split has improved loss 0.0010626803920600336
Column EDUCATION2 split has improved loss 0.0010626803920600336
Column EDUCATION3 split has improved loss 4.5160851918912837e-05
Column MARRIAGE0 split has improved loss 9.449539629236003e-05
Column MARRIAGE1 split has improved loss 0.0003235515454700355
Column MARRIAGE2 split has improved loss 0.0003235515454700355
Column MARRIAGE3 split has improved loss 3.5359652165056765e-05
Column AGE0 split has imp

('LIMIT_BAL1', 0.32597885804022653)

In [187]:
find_split(train_x_t, train_y, acc, verbose = True)

Column LIMIT_BAL0 split has improved loss 0.0
Column LIMIT_BAL1 split has improved loss 0.0
Column LIMIT_BAL2 split has improved loss 0.0
Column LIMIT_BAL3 split has improved loss 0.0
Column SEX0 split has improved loss nan
Column SEX1 split has improved loss 0.0
Column SEX2 split has improved loss 0.0
Column SEX3 split has improved loss 0.0
Column EDUCATION0 split has improved loss 0.0
Column EDUCATION1 split has improved loss 0.0
Column EDUCATION2 split has improved loss 0.0
Column EDUCATION3 split has improved loss 0.0
Column MARRIAGE0 split has improved loss 0.0
Column MARRIAGE1 split has improved loss 0.0
Column MARRIAGE2 split has improved loss 0.0
Column MARRIAGE3 split has improved loss 0.0
Column AGE0 split has improved loss 0.0
Column AGE1 split has improved loss 0.0
Column AGE2 split has improved loss 0.0
Column AGE3 split has improved loss 0.0
Column PAY_00 split has improved loss 0.0
Column PAY_01 split has improved loss 0.0
Column PAY_02 split has improved loss 0.0
Column

(None, 0.77688000000000001)

In [188]:
np.mean(train_y[train_x_t['LIMIT_BAL1']])

0.16503548748599178

In [172]:
np.mean(train_y[~train_x_t['LIMIT_BAL1']])

0.29005596211795093

In [173]:
np.mean(train_y[train_x_t['AGE1']])

0.22458921041151664

In [174]:
np.mean(train_y[~train_x_t['AGE1']])

0.22132313711541882

In [236]:
#Slow but simple
class decision_tree(object):
    def fit(self, x,y,depth=5,loss=MSE, minsize = 1, quintiles = 2):
        mu = np.mean(y)
        self.f = lambda x: mu
        if(len(x)<=minsize or depth == 0):
            return 
        tr = bin_transformer(x, quintiles)
        tr_x, fns = tr.transform(x)
        split, _ = find_split(tr_x,y,loss)
        if split == None:
            return
        splitter = fns[split]
        mask = tr_x[split]
        left = decision_tree()
        right = decision_tree()
        left.fit(tr_x[~mask],y[~mask],depth-1,loss, minsize, quintiles)
        right.fit(tr_x[mask],y[mask],depth-1,loss, minsize, quintiles)
        def g(x):
            print("spliting at {}, {}".format(split,x))
            if(splitter(x)):
                print("right {}".format(right.f(x)))
                return right.f(x)
            else:
                print("right {}".format(right.f(x)))
                return left.f(x)
        self.f = g
  
        
        
            
        
        

In [237]:
dt = decision_tree()

In [238]:
dt.fit(train_x, train_y)

In [248]:
dt.f(train_x.iloc[0,:]), train_y.iloc[0]

spliting at LIMIT_BAL0, LIMIT_BAL    20000.0
SEX              2.0
EDUCATION        2.0
MARRIAGE         1.0
AGE             24.0
PAY_0            2.0
PAY_2            2.0
PAY_3           -1.0
PAY_4           -1.0
PAY_5           -2.0
PAY_6           -2.0
BILL_AMT1     3913.0
BILL_AMT2     3102.0
BILL_AMT3      689.0
BILL_AMT4        0.0
BILL_AMT5        0.0
BILL_AMT6        0.0
PAY_AMT1         0.0
PAY_AMT2       689.0
PAY_AMT3         0.0
PAY_AMT4         0.0
PAY_AMT5         0.0
PAY_AMT6         0.0
Name: 0, dtype: float64


TypeError: 'tuple' object is not callable

In [245]:
train_x.iloc[0,:]

LIMIT_BAL    20000.0
SEX              2.0
EDUCATION        2.0
MARRIAGE         1.0
AGE             24.0
PAY_0            2.0
PAY_2            2.0
PAY_3           -1.0
PAY_4           -1.0
PAY_5           -2.0
PAY_6           -2.0
BILL_AMT1     3913.0
BILL_AMT2     3102.0
BILL_AMT3      689.0
BILL_AMT4        0.0
BILL_AMT5        0.0
BILL_AMT6        0.0
PAY_AMT1         0.0
PAY_AMT2       689.0
PAY_AMT3         0.0
PAY_AMT4         0.0
PAY_AMT5         0.0
PAY_AMT6         0.0
Name: 0, dtype: float64